# Data Quality Testing - Workshop SDS25

## Goal
The Goal of this Workshop is gaining hands-on experience in Data Quality assessment, using the Great Expectations Framework. However, the knowledge gained in this Workshop is transferrable to other Testing Frameworks as well. The Data Quality Assessment gives you confidence in the data foundation used in AI.

## DataSet
The DataSet we will be using during this Workshop is the "Bike Sharing" Dataset (https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset). It contains aggregated information about bike rental, either per hour or per day, over two years, in Washington D.C.

The DataSet was already loaded into the database in this container (database.db). For each set of the exercise, the required data is loaded with a corresponding code cell, so you don't have to worry about it.

The Notebook will lead you through the following scenarios:  
- Making basic expectations based on Spring Data of the first year
- Checking and refining expectations based on added Summar Data of the first year
- Same for Autumn, using increasingly complex Expectations
- Same for Winter
- Finally using the data from year 2 to verify the expectations, identify datashifts etc.

In the end, the goal is to have a solid data foundation to implement a flexible pricing system.



# Task 1: Inspect the DataSet
Familiarize yourself with the "Bike Sharing" Dataset (https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset). Explore the dataset using the cells below.

In [1]:
import database
import great_expectations as gx
import sqlite3
import pandas as pd

metric column.standard_deviation.aggregate_fn is being registered with different metric_provider; overwriting metric_provider


In [18]:
# Initialise the dataset
database.init()

conn = sqlite3.connect('database.db')

2025-04-08 15:08:57.530 | INFO     | database:reset_database:54 - Database reset completed
2025-04-08 15:08:57.531 | INFO     | database:init:142 - Initializing database to step 0: 0_spring_2011


2025-04-08 15:08:57.533 | INFO     | database:apply_migration:88 - Migration 0_create_table.sql from 0_spring_2011 - Successfully applied
2025-04-08 15:08:57.552 | INFO     | database:apply_migration:88 - Migration 1_bike_rental_2011_spring.sql from 0_spring_2011 - Successfully applied


In [15]:
database.set_step(3)
pd.read_sql_query("SELECT * FROM bike_rental;", conn)

2025-04-08 15:07:25.024 | INFO     | database:set_step:221 - Processing directory: 0_spring_2011
2025-04-08 15:07:25.026 | DEBUG    | database:set_step:228 - Skipping 0_create_table.sql (already applied)
2025-04-08 15:07:25.026 | DEBUG    | database:set_step:228 - Skipping 1_bike_rental_2011_spring.sql (already applied)
2025-04-08 15:07:25.027 | INFO     | database:set_step:221 - Processing directory: 1_summer_2011
2025-04-08 15:07:25.028 | DEBUG    | database:set_step:228 - Skipping 0_bike_rental_2011_summer.sql (already applied)
2025-04-08 15:07:25.030 | INFO     | database:set_step:221 - Processing directory: 2_fall_2011
2025-04-08 15:07:25.031 | DEBUG    | database:set_step:228 - Skipping 0_bike_rental_2011_fall.sql (already applied)
2025-04-08 15:07:25.032 | INFO     | database:set_step:221 - Processing directory: 3_winter_2011
2025-04-08 15:07:25.033 | DEBUG    | database:set_step:228 - Skipping 0_bike_rental_2011_winter.sql (already applied)


,id,dteday,season,yr,mnth,hour,holiday,weekday,workingday,weathersit,...,atemp,hum,windspeed,casual,registered,total,felt_temp,humidity,year,weather
0,1808,2011-03-21,Spring,None,3,0,0,Monday,1,None,...,None,None,26.0027,2,11,13,3.9980,66.0,2011,Bad
1,1809,2011-03-21,Spring,None,3,1,0,Monday,1,None,...,None,None,26.0027,1,6,7,3.9980,71.0,2011,Okay
2,1810,2011-03-21,Spring,None,3,2,0,Monday,1,None,...,None,None,22.0028,1,5,6,3.9980,71.0,2011,Okay
3,1811,2011-03-21,Spring,None,3,3,0,Monday,1,None,...,None,None,22.0028,0,1,1,3.9980,71.0,2011,Okay
4,1812,2011-03-21,Spring,None,3,5,0,Monday,1,None,...,None,None,19.9995,1,1,2,3.9980,81.0,2011,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,8641,2011-12-31,Winter,None,12,19,0,Saturday,0,None,...,None,None,15.0013,19,73,92,11.9972,54,2011,Good
8641,8642,2011-12-31,Winter,None,12,20,0,Saturday,0,None,...,None,None,15.0013,8,63,71,11.9972,54,2011,Good
8642,8643,2011-12-31,Winter,None,12,21,0,Saturday,0,None,...,None,None,12.9980,2,50,52,11.0006,58.0,2011,Good
8643,8644,2011-12-31,Winter,None,12,22,0,Saturday,0,None,...,None,None,8.9981,2,36,38,9.9974,62,2011,Good


In [16]:
# Set the context for great expectations (the data set)
context = gx.get_context()

data_source = context.data_sources.add_sqlite("sample", connection_string="sqlite:///database.db")

In [17]:
# Get the batch
asset_name = "bike_rental"
database_table_name = "bike_rental"
table_data_asset = data_source.add_table_asset(
    table_name=database_table_name, name=asset_name
)

full_table_batch_definition = table_data_asset.add_batch_definition_whole_table(
    name="0_spring_2011",
)


In [5]:
# TODO: fix something in database, atemp, humidity and weathersit are None, also the year is None => probably integer and double values were not transported correctly?

In [6]:
full_table_batch = full_table_batch_definition.get_batch()
full_table_batch.head()


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

     id      dteday  season    yr  mnth  hour  holiday weekday  workingday  \
0  1808  2011-03-21  Spring  None     3     0        0  Monday           1   
1  1809  2011-03-21  Spring  None     3     1        0  Monday           1   
2  1810  2011-03-21  Spring  None     3     2        0  Monday           1   
3  1811  2011-03-21  Spring  None     3     3        0  Monday           1   
4  1812  2011-03-21  Spring  None     3     5        0  Monday           1   

  weathersit  ...  atemp   hum windspeed  casual  registered  total  \
0       None  ...   None  None   26.0027       2          11     13   
1       None  ...   None  None   26.0027       1           6      7   
2       None  ...   None  None   22.0028       1           5      6   
3       None  ...   None  None   22.0028       0           1      1   
4       None  ...   None  None   19.9995       1           1      2   

   felt_temp  humidity  year weather  
0      3.998      66.0  2011     Bad  
1      3.998      71.0  20

In [23]:
# Read the data from the database using SQL
query = """
SELECT * FROM bike_rental
WHERE casual = 2
"""

pd.read_sql_query(query, conn)

,id,dteday,season,yr,mnth,hour,holiday,weekday,workingday,weathersit,...,atemp,hum,windspeed,casual,registered,total,felt_temp,humidity,year,weather
0,1808,2011-03-21,Spring,None,3,0,0,Monday,1,None,...,None,None,26.0027,2,11,13,3.9980,66.0,2011,Bad
1,1813,2011-03-21,Spring,None,3,6,0,Monday,1,None,...,None,None,16.9979,2,30,32,3.9980,76.0,2011,Bad
2,1837,2011-03-22,Spring,None,3,6,0,Tuesday,1,None,...,None,None,16.9979,2,58,60,11.0006,87.0,2011,Good
3,1877,2011-03-23,Spring,None,3,23,0,Wednesday,1,None,...,None,None,0.0000,2,24,26,5.9978,90.0,2011,Bad
4,1885,2011-03-24,Spring,None,3,7,0,Thursday,1,None,...,None,None,15.0013,2,106,108,1.0016,100.0,2011,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,3824,2011-06-13,Spring,None,6,5,0,Monday,1,None,...,None,None,26.0027,2,27,29,18.0032,64.0,2011,Good
82,3845,2011-06-14,Spring,None,6,2,0,Tuesday,1,None,...,None,None,19.0012,2,8,10,24.9992,49.0,2011,Good
83,3914,2011-06-16,Spring,None,6,23,0,Thursday,1,None,...,None,None,16.9979,2,14,16,20.9996,83.0,2011,Bad
84,3917,2011-06-17,Spring,None,6,2,0,Friday,1,None,...,None,None,7.0015,2,11,13,18.9998,94.0,2011,Good


# Task 2: Set Expectations for the Spring Data
Now that you explored the dataset, it is time to set up your first expectations (exciting!)

# TODO: add some basics expectations as a markdown for example

In [ ]:
# TODO: give one example of how to set expectations for a datatype


In [ ]:
# TODO: make them set one expectation for another datatype themselves

In [ ]:
# TODO: add expectations of maximum bike rentals according to the max today + 50 or so

In [ ]:
# TODO: set expectation for correlation of rising temperatures to rising bike rentals

In [ ]:
# TODO: check the expectations

But, you didn't come to this Workshop just to see a fancy way of doing exactly the same as your basic unit test is doing, so let's get into more complex stuff

# Task 3: Adjust and Set New Expectations for the Summer Data

In [ ]:
# TODO: load the summer dataset

In [ ]:
# TODO: run the expectations for that new dataset and look at the output

## Excursion: Data Docs

In [ ]:
# TODO: explain data docs and how it works

In [ ]:
# TODO: generate the data docs

In [ ]:
# TODO: look at the data docs

## Back To Business

In [6]:
# TODO: refine the expectations

In [7]:
# TODO: add more complex expectations (give them a list of suggestions again)

In [ ]:
# TODO: add a fun expectation, that expects bike rentals to rise, because they have risen before

In [8]:
# TODO: check the expectations

# Task 4: Adjust for Autumn 

In [ ]:
# TODO: load new dataset

In [ ]:
# TODO: check the expectations

In [ ]:
# TODO: fix what needs fixing

In [ ]:
# TODO: Maybe add something even more complex?

In [ ]:
# TODO: Recheck the Expectations

# Task 5: Check with Winter and set final expectations
You can check out all kinds of expectations here: https://greatexpectations.io/expectations/

In [ ]:
# TODO: load new dataset

In [ ]:
# TODO: check the expectations

In [ ]:
# TODO: fix what needs fixing

In [ ]:
# TODO: Maybe add something even more complex?

In [ ]:
# TODO: Recheck the Expectations

# Task 6: Verify your data and see if something shifts the next year

In [ ]:
# TODO: load new dataset

In [ ]:
# TODO: check the expectations

Discuss these expectations => did you do a good job? What changed? Do you now have confidence in your data foundation for your AI model? Discuss pros and cons of using a Testing Framework!

# Task 7: Think about AI Implementation

Could you now implement AI to design a flexible pricing model? How would you do it? What is the advantage over doing this by hand?

In [9]:
# TODO: Make this last part better and more to the point ^^